In [ ]:
# Data Crawler
# gopax 거래소에서 받고싶은 정보의 시간 범위를 작성하면 종가를 csv파일로 저장해주는 프로그램

In [1]:
import gopax_api_wrapper as gopax
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
#data frame을 받기
def get_past_dataframe(trading_pair, start, end, interval):
    start_ts = gopax.get_timestamp(start)
    end_ts = gopax.get_timestamp(end)
    raw_result = gopax.get_past_data(trading_pair, start_ts, end_ts, interval)
    result = pd.DataFrame(raw_result, columns = ['time', 'low', 'high', 'open', 'close', 'volume'])
    return result

In [8]:
#시작 시간 지정
#UTC시간으로 변경시키는 과정

from time import mktime, strptime
time_format = '%Y-%m-%d %H:%M:%S'
time_tuple = strptime('2018-03-01 10:00:00', time_format)
utc_start = mktime(time_tuple)

#마지막 시간 지정
#UTC시간으로 변경시키는 과정

time_format = '%Y-%m-%d %H:%M:%S'
time_tuple = strptime('2018-04-01 10:00:00', time_format)
utc_end = mktime(time_tuple)

In [9]:
#거래소에서 불러올 시간 간격 설정

start_utc_time = utc_start      #2018-01-01 10:00:00 시작
start_end_term = 60*60*24*7    #시작-마지막 간격 = 1주일
next_data_term = 86400         #train 데이터간 시간 간격 = 1일
time_term = 30                 #데이터간 시간 간격 = 30분

label_start_end_term = 60*60   #lable 데이터 시작 마지막 간격

end_utc_time = utc_end
how_long = int((utc_end - utc_start) / (3600*24))


In [11]:
#백테스팅
# how_long에 저장할 개수를 지정
#df_train 딕셔너리에 df_train_0 이런 식으로 데이터 저장하기
#18년 1월 1일 부터 시작해서 일주일간의 데이터 적용

df_train = {}
value_sum = np.array(())

for how_long in range(how_long):
    df_train["df_train_"+str(how_long)] = \
    get_past_dataframe('BTC-KRW', str(datetime.utcfromtimestamp(start_utc_time+how_long*next_data_term)), 
                       str(datetime.utcfromtimestamp(start_utc_time+start_end_term+how_long*next_data_term)), time_term)
    
    if how_long == 0:
        value = df_train["df_train_"+str(0)]["close"].values[:300]
    else:
        value= np.vstack([value, df_train["df_train_"+str(how_long)]["close"].values[:300]]) 
    
    #train데이터를 exel로 옮기기
a = np.asarray(value)
np.savetxt("train_1.csv", a, delimiter=",")

KeyError: 'df_train_0'